In [51]:
# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from npj_utils import *
from sklearn.multioutput import MultiOutputClassifier
import xgboost as xgb

In [53]:
mimic_iv_path = "/data/wang/junh/datasets/physionet.org/files/mimiciv/2.2"
mm_dir = "/data/wang/junh/datasets/multimodal"

output_dir = os.path.join(mm_dir, "preprocessing")

In [59]:
import pickle

base_name = "pheno" # ihm, los, pheno

if "pheno" in base_name:
    base_name += "-all"
else:
    base_name += "-48"

# base_name += "-cxr-notes-ecg-missingInd"
base_name += "-cxr-notes-missingInd"

f_path = os.path.join(output_dir, f"train_{base_name}_stays.pkl")

with open(f_path, "rb") as f:
    train_stays = pickle.load(f)

f_path = os.path.join(output_dir, f"val_{base_name}_stays.pkl")

with open(f_path, "rb") as f:
    val_stays = pickle.load(f)

f_path = os.path.join(output_dir, f"test_{base_name}_stays.pkl")

with open(f_path, "rb") as f:
    test_stays = pickle.load(f)

In [60]:
include_notes = True
include_cxr = True
include_ecg = False

In [61]:
X_train = calc_ts_embeddings(train_stays)

if include_notes:
    txt_df = calc_avg_text_embedding(train_stays)
    X_train = pd.concat([X_train, txt_df], axis=1)

if include_cxr:
    cxr_df = calc_avg_cxr_embedding(train_stays) 
    X_train = pd.concat([X_train, cxr_df], axis=1)

if include_ecg:
    ecg_df = calc_avg_ecg_embedding(train_stays)
    X_train = pd.concat([X_train, ecg_df], axis=1)

y_train = extract_labels(train_stays)

col_names = X_train.columns

Calculating CXR Embeddings: 100%|██████████| 51213/51213 [6:28:57<00:00,  2.19it/s]  


In [43]:
X_test = calc_ts_embeddings(test_stays)

if include_notes:
    txt_df = calc_avg_text_embedding(test_stays)
    X_test = pd.concat([X_test, txt_df], axis=1)

if include_cxr:
    cxr_df = calc_avg_cxr_embedding(test_stays)
    X_test = pd.concat([X_test, cxr_df], axis=1)

if include_ecg:
    ecg_df = calc_avg_ecg_embedding(test_stays)
    X_test = pd.concat([X_test, ecg_df], axis=1)

y_test = extract_labels(test_stays)

Calculating Text Embeddings:  50%|████▉     | 2608/5259 [00:26<01:08, 38.50it/s] 

Calculating CXR Embeddings: 100%|██████████| 5259/5259 [03:06<00:00, 28.22it/s] 


In [44]:
X_val = calc_ts_embeddings(val_stays)

if include_notes:
    txt_df = calc_avg_text_embedding(val_stays)
    X_val = pd.concat([X_val, txt_df], axis=1)

if include_cxr:
    cxr_df = calc_avg_cxr_embedding(val_stays)
    X_val = pd.concat([X_val, cxr_df], axis=1)

if include_ecg:
    ecg_df = calc_avg_ecg_embedding(val_stays)
    X_val = pd.concat([X_val, ecg_df], axis=1)

y_val = extract_labels(val_stays)

Calculating Time Series Embeddings:  60%|██████    | 3166/5262 [00:06<00:04, 504.90it/s]

Calculating CXR Embeddings: 100%|██████████| 5262/5262 [03:12<00:00, 27.40it/s] 


In [45]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [55]:
from prediction_util import run_xgb, run_xgb_multilabel

seed = 1
if "pheno" in base_name:
    y_pred_test, y_pred_prob_test, y_pred_train, y_pred_prob_train, gs = run_xgb_multilabel(X_train, y_train, X_test, gpu=0, seed=seed, n_jobs=16)
else:
    y_pred_test, y_pred_prob_test, y_pred_train, y_pred_prob_train, gs = run_xgb(X_train, y_train, X_test, gpu=0, seed=seed, n_jobs=16)

ValueError: y must have at least two dimensions for multi-output regression but has only one.

# Pheno (no ECG)

In [62]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(51213, 2122) (51213, 25) (5259, 2122) (5259,)


In [63]:
print(y_pred_test.shape)
print(y_pred_prob_test.shape)
print(y_pred_train.shape)
print(y_pred_prob_train.shape)
print(gs.best_params_)

(5259,)
(5259, 2)
(24557,)
(24557, 2)
{'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 200}


# LOS (no ECG)

In [47]:
print(y_pred_test.shape)
print(y_pred_prob_test.shape)
print(y_pred_train.shape)
print(y_pred_prob_train.shape)
print(gs.best_params_)

(5259,)
(5259,)
(24557,)
(24557,)
{'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 200}


In [48]:
est = xgb.XGBClassifier(verbosity=2, seed=seed,eval_metric='logloss', n_jobs=32,learning_rate= 0.05, max_depth= 6, n_estimators= 200, device="cuda")
est.fit(X_train, y_train)

y_pred_test = est.predict(X_test)
y_pred_prob_test = est.predict_proba(X_test)

# Evaluate
_ = evaluate_model(y_test, y_pred_test, y_pred_prob_test[:,1])

Proportion of outcome: 0.4738543449324967
F1 Score: 0.7221087803972527
Accuracy: 0.7153451226468911
Balanced Accuracy: 0.7185827318180737
AUC: 0.7899929575871557
AUPRC: 0.7317035269399694
Confusion Matrix: [[1817  950]
 [ 547 1945]]


In [49]:
# Make predictions on the training set
y_pred_prob_train = est.predict_proba(X_train)
y_pred_train = est.predict(X_train)

# F1 Score for the training set
f1_train = f1_score(y_train, y_pred_train)
print(f"F1 Score for Training Set is: {f1_train}")

# Accuracy for the training set
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy for Training Set is: {accuracy_train}")

# Balanced Accuracy for the training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_pred_train)
print(f"Balanced Accuracy for Training Set is: {balanced_accuracy_train}")

# AUC for the training set
# Ensure that y_pred_prob_train[:, 1] exists if it's a binary classification
if len(np.unique(y_train)) == 2:
    auc_train = roc_auc_score(y_train, y_pred_prob_train[:, 1])
    print(f"AUC for Training Set is: {auc_train}")
else:
    print("AUC is typically used for binary classification. Adjust the code for multi-class if needed.")

# AUPRC for the training set
if len(np.unique(y_train)) == 2:
    auprc_train = average_precision_score(y_train, y_pred_prob_train[:, 1])
    print(f"AUPRC for Training Set is: {auprc_train}")
else:
    print("AUPRC is typically used for binary classification. Adjust the code for multi-class if needed.")

# Confusion Matrix for the training set
cm_train = confusion_matrix(y_train, y_pred_train)
print(f"Confusion Matrix for Training Set is: \n{cm_train}")


F1 Score for Training Set is: 0.82063233123592
Accuracy for Training Set is: 0.8119477134829173
Balanced Accuracy for Training Set is: 0.8149546634839877
AUC for Training Set is: 0.8971280934763423
AUPRC for Training Set is: 0.8722761350014441
Confusion Matrix for Training Set is: 
[[ 9375  3369]
 [ 1249 10564]]


In [50]:
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score, average_precision_score, confusion_matrix

# Assumed correction: Ensure y_pred_prob_test is fetched correctly and has two columns
# y_pred_prob_test = model.predict_proba(X_test)  # Correctly fetching test probabilities

# F1 Score
f1 = f1_score(y_test, y_pred_test)
print(f"F1 Score for Testing Set is: {f1}")

# Accuracy
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy for Testing Set is: {accuracy}")

# Balanced Accuracy
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print(f"Balanced Accuracy for Testing Set is: {balanced_accuracy}")

# AUC
if y_pred_prob_test.ndim == 2 and y_pred_prob_test.shape[1] == 2:  # Ensuring binary classification with two probability outputs
    auc = roc_auc_score(y_test, y_pred_prob_test[:, 1])
    print(f"AUC for Testing Set is: {auc}")
else:
    print("Error: Expected binary class probabilities for AUC calculation.")

# AUPRC
if y_pred_prob_test.ndim == 2 and y_pred_prob_test.shape[1] == 2:
    auprc = average_precision_score(y_test, y_pred_prob_test[:, 1])
    print(f"AUPRC for Testing Set is: {auprc}")
else:
    print("Error: Expected binary class probabilities for AUPRC calculation.")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)
print(f"Confusion Matrix for Testing Set is: \n{cm}")

F1 Score for Testing Set is: 0.7221087803972527
Accuracy for Testing Set is: 0.7153451226468911
Balanced Accuracy for Testing Set is: 0.7185827318180737
AUC for Testing Set is: 0.7899929575871557
AUPRC for Testing Set is: 0.7317035269399694
Confusion Matrix for Testing Set is: 
[[1817  950]
 [ 547 1945]]


# IHM (no ECG)

In [11]:
print(y_pred_test.shape)
print(y_pred_prob_test.shape)
print(y_pred_train.shape)
print(y_pred_prob_train.shape)
print(gs.best_params_)


(5259,)
(5259,)
(24557,)
(24557,)
{'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 200}


In [12]:
print("Sample probabilities for training:", y_pred_prob_train[:5])
print("Sample probabilities for testing:", y_pred_prob_test[:5])


Sample probabilities for training: [0.53186744 0.5715928  0.50406665 0.46213225 0.28154257]
Sample probabilities for testing: [0.12913318 0.3875949  0.16137534 0.12450095 0.20820318]


In [ ]:
est = MultiOutputClassifier(xgb.XGBClassifier(verbosity=2, seed=42,
                                                  tree_method='gpu_hist', gpu_id=1,
                                                  eval_metric='logloss', n_jobs=32,learning_rate= 0.05, max_depth= 5, n_estimators= 200))
est.fit(X_train, y_train)
y_pred_prob_test = est.predict_proba(X_test)
y_pred_test = est.predict(X_test)
# Evaluate
_ = evaluate_model(y_test, y_pred_test, y_pred_prob_test)

In [31]:
est = xgb.XGBClassifier(verbosity=2, seed=seed,eval_metric='logloss', n_jobs=32,learning_rate= 0.05, max_depth= 5, n_estimators= 200, device="cuda")
est.fit(X_train, y_train)

y_pred_test = est.predict(X_test)
y_pred_prob_test = est.predict_proba(X_test)

# Evaluate
_ = evaluate_model(y_test, y_pred_test, y_pred_prob_test[:,1])

Proportion of outcome: 0.1418520631298726
F1 Score: 0.2694300518134715
Accuracy: 0.8659440958357102
Balanced Accuracy: 0.5772709640128102
AUC: 0.8038226178885067
AUPRC: 0.4391488439113681
Confusion Matrix: [[4424   89]
 [ 616  130]]


In [32]:
# Make predictions on the training set
y_pred_prob_train = est.predict_proba(X_train)
y_pred_train = est.predict(X_train)

# F1 Score for the training set
f1_train = f1_score(y_train, y_pred_train)
print(f"F1 Score for Training Set is: {f1_train}")

# Accuracy for the training set
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy for Training Set is: {accuracy_train}")

# Balanced Accuracy for the training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_pred_train)
print(f"Balanced Accuracy for Training Set is: {balanced_accuracy_train}")

# AUC for the training set
# Ensure that y_pred_prob_train[:, 1] exists if it's a binary classification
if len(np.unique(y_train)) == 2:
    auc_train = roc_auc_score(y_train, y_pred_prob_train[:, 1])
    print(f"AUC for Training Set is: {auc_train}")
else:
    print("AUC is typically used for binary classification. Adjust the code for multi-class if needed.")

# AUPRC for the training set
if len(np.unique(y_train)) == 2:
    auprc_train = average_precision_score(y_train, y_pred_prob_train[:, 1])
    print(f"AUPRC for Training Set is: {auprc_train}")
else:
    print("AUPRC is typically used for binary classification. Adjust the code for multi-class if needed.")

# Confusion Matrix for the training set
cm_train = confusion_matrix(y_train, y_pred_train)
print(f"Confusion Matrix for Training Set is: \n{cm_train}")


F1 Score for Training Set is: 0.515621767018415
Accuracy for Training Set is: 0.9046707659730423
Balanced Accuracy for Training Set is: 0.6770478968464135
AUC for Training Set is: 0.9052444653528526
AUPRC for Training Set is: 0.7249747341606338
Confusion Matrix for Training Set is: 
[[20970   127]
 [ 2214  1246]]


In [33]:
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score, average_precision_score, confusion_matrix

# Assumed correction: Ensure y_pred_prob_test is fetched correctly and has two columns
# y_pred_prob_test = model.predict_proba(X_test)  # Correctly fetching test probabilities

# F1 Score
f1 = f1_score(y_test, y_pred_test)
print(f"F1 Score for Testing Set is: {f1}")

# Accuracy
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy for Testing Set is: {accuracy}")

# Balanced Accuracy
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print(f"Balanced Accuracy for Testing Set is: {balanced_accuracy}")

# AUC
if y_pred_prob_test.ndim == 2 and y_pred_prob_test.shape[1] == 2:  # Ensuring binary classification with two probability outputs
    auc = roc_auc_score(y_test, y_pred_prob_test[:, 1])
    print(f"AUC for Testing Set is: {auc}")
else:
    print("Error: Expected binary class probabilities for AUC calculation.")

# AUPRC
if y_pred_prob_test.ndim == 2 and y_pred_prob_test.shape[1] == 2:
    auprc = average_precision_score(y_test, y_pred_prob_test[:, 1])
    print(f"AUPRC for Testing Set is: {auprc}")
else:
    print("Error: Expected binary class probabilities for AUPRC calculation.")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)
print(f"Confusion Matrix for Testing Set is: \n{cm}")


F1 Score for Testing Set is: 0.2694300518134715
Accuracy for Testing Set is: 0.8659440958357102
Balanced Accuracy for Testing Set is: 0.5772709640128102
AUC for Testing Set is: 0.8038226178885067
AUPRC for Testing Set is: 0.4391488439113681
Confusion Matrix for Testing Set is: 
[[4424   89]
 [ 616  130]]


In [34]:
est.feature_importances_

# Get the top 10 most important features
indices = np.argsort(est.feature_importances_)[::-1]
top_indices = indices[:100]
print('Feature ranking:')
for i in range(50):
    print('%d. %s (%f)' % (i + 1, col_names[top_indices[i]], est.feature_importances_[top_indices[i]]))

Feature ranking:
1. Anion gap_max (0.083074)
2. Bicarbonate_max (0.017980)
3. Bicarbonate_maxdiff (0.009172)
4. te_130 (0.008616)
5. Anion gap_min (0.008259)
6. Bicarbonate_min (0.007819)
7. GCS - Eye Opening_min (0.007582)
8. te_201 (0.006392)
9. Alkaline Phosphate_max (0.006281)
10. Anion gap_trend (0.006204)
11. te_280 (0.006129)
12. Diastolic BP_mean (0.005578)
13. te_52 (0.005152)
14. te_665 (0.005065)
15. te_76 (0.005051)
16. GCS - Eye Opening_mean (0.004789)
17. Anion gap_sumabsdiff (0.004686)
18. te_734 (0.004618)
19. Alkaline Phosphate_mean (0.004582)
20. Anion gap_mean (0.004333)
21. Creatinine_variance (0.004297)
22. Bicarbonate_trend (0.004241)
23. te_147 (0.004133)
24. te_319 (0.004056)
25. Alkaline Phosphate_trend (0.004046)
26. te_73 (0.003970)
27. Bicarbonate_mean (0.003799)
28. Creatinine_min (0.003569)
29. te_235 (0.003500)
30. te_25 (0.003483)
31. te_357 (0.003422)
32. Chloride_mean (0.003335)
33. te_760 (0.003254)
34. Alkaline Phosphate_min (0.003224)
35. te_109 (0.

In [35]:
# print("TRAIN")
# _ = evaluate_model(y_train, y_pred_train, y_pred_prob_train)

print(f"Task: {base_name}")
print(f"Seed: {seed}")

modals = "ts"

if include_notes:
    modals += "+text"

if include_cxr:
    modals += "+cxr"

if include_ecg:
    modals += "+ecg"

print(f"Modals: {modals}")

print("\n\nTEST")
_ = evaluate_model(y_test, y_pred_test, y_pred_prob_test)

Task: ihm-48-cxr-notes-missingInd
Seed: 1
Modals: ts+text+cxr


TEST


ValueError: y should be a 1d array, got an array of shape (5259, 2) instead.